In [1]:
%%writefile predictor.py

import os

import numpy as np
import joblib
import pandas as pd

class ChurnPredictor(object):

    _COLUMN_NAMES=['CLIENTNUM','Customer_Age','Gender','Dependent_count','Education_Level','Marital_Status','Income_Category','Card_Category','Months_on_book','Total_Relationship_Count','Months_Inactive_12_mon','Contacts_Count_12_mon','Credit_Limit','Total_Revolving_Bal','Avg_Open_To_Buy','Total_Amt_Chng_Q4_Q1','Total_Trans_Amt','Total_Trans_Ct','Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio']

    def __init__(self, model):
        self._model = model
        #self.col=[]

    def predict(self, instances, **kwargs):
        inputs = pd.DataFrame(data=[instances], columns=self._COLUMN_NAMES)
        outputs = self._model.predict(inputs)
        return outputs.tolist()

    @classmethod
    def from_path(cls, model_dir):
        model_path = os.path.join(model_dir, 'model.joblib')
        model = joblib.load(model_path)
        #self._COLUMN_NAMES=['CLIENTNUM','Customer_Age','Gender','Dependent_count','Education_Level','Marital_Status','Income_Category','Card_Category','Months_on_book','Total_Relationship_Count','Months_Inactive_12_mon','Contacts_Count_12_mon','Credit_Limit','Total_Revolving_Bal','Avg_Open_To_Buy','Total_Amt_Chng_Q4_Q1','Total_Trans_Amt','Total_Trans_Ct','Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio']
        return cls(model)

Writing predictor.py


In [2]:
from predictor import ChurnPredictor

col_names=['CLIENTNUM','Customer_Age','Gender','Dependent_count','Education_Level','Marital_Status','Income_Category','Card_Category','Months_on_book','Total_Relationship_Count','Months_Inactive_12_mon','Contacts_Count_12_mon','Credit_Limit','Total_Revolving_Bal','Avg_Open_To_Buy','Total_Amt_Chng_Q4_Q1','Total_Trans_Amt','Total_Trans_Ct','Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio']
model = ChurnPredictor.from_path('.')
#print(model._model)
instance = [768805383,45,'M','3','Graduate','Married','Unknown','Blue',39,5,1,3,12691,777,11914,1.335,1144,42,1.625,0.061]

print(model.predict(instance))

[1]


In [3]:
%%writefile setup.py
from setuptools import setup
from setuptools import find_packages

REQUIRED_PACKAGES = ['xgboost','scikit-learn']

setup(
    name='custom_predict',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    scripts=['predictor.py'])

Writing setup.py


In [4]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
creating custom_predict.egg-info
writing custom_predict.egg-info/PKG-INFO
writing dependency_links to custom_predict.egg-info/dependency_links.txt
writing requirements to custom_predict.egg-info/requires.txt
writing top-level names to custom_predict.egg-info/top_level.txt
writing manifest file 'custom_predict.egg-info/SOURCES.txt'
reading manifest file 'custom_predict.egg-info/SOURCES.txt'
writing manifest file 'custom_predict.egg-info/SOURCES.txt'

running check


creating custom_predict-0.1
creating custom_predict-0.1/custom_predict.egg-info
copying files to custom_predict-0.1...
copying predictor.py -> custom_predict-0.1
copying setup.py -> custom_predict-0.1
copying custom_predict.egg-info/PKG-INFO -> custom_predict-0.1/custom_predict.egg-info
copying custom_predict.egg-info/SOURCES.txt -> custom_predict-0.1/custom_predict.egg-info
copying custom_predict.egg-info/dependency_links.txt -> custom_predict-0.1/custom_predict.egg-info
copying custom_predict

In [5]:
!gsutil cp ./dist/custom_predict-0.1.tar.gz gs://aiplatform_irisgcp/custom_predict-0.1.tar.gz


Copying file://./dist/custom_predict-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.2 KiB/  1.2 KiB]                                                
Operation completed over 1 objects/1.2 KiB.                                      


In [6]:
!gcloud beta ai-platform models create ChurnPredictor --regions us-central1 --enable-console-logging

Using endpoint [https://ml.googleapis.com/]
Created ai platform model [projects/machine-learning-348010/models/ChurnPredictor].


In [7]:
!sudo apt-get install google-cloud-sdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-python
  google-cloud-sdk-pubsub-emulator google-cloud-sdk-bigtable-emulator
  google-cloud-sdk-datastore-emulator
The following packages will be upgraded:
  google-cloud-sdk
1 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
Need to get 130 MB of archives.
After this operation, 316 kB disk space will be freed.
Get:1 http://packages.cloud.google.com/apt cloud-sdk-buster/main amd64 google-cloud-sdk all 384.0.1-0 [130 MB]
Fetched 130 MB in 2s (58.0 MB/s)           
(Reading database ... 121921 files and directories currently installed.)
Preparing to unpack .../google-cloud-sdk_384.0.1-0_all.deb ...
Unpacking google-cloud-sdk (384.0.1-0) over (384.0.0-0) ...
Setting up google-cloud-sdk (384.0.1-0) ...
Processing triggers for man-db (2.8.5-2) ...


In [8]:
!gcloud components list


Your current Google Cloud CLI version is: 384.0.1
The latest available version is: 384.0.1

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                 Components                                                 │
├───────────────┬──────────────────────────────────────────────────────┬──────────────────────────┬──────────┤
│     Status    │                         Name                         │            ID            │   Size   │
├───────────────┼──────────────────────────────────────────────────────┼──────────────────────────┼──────────┤
│ Not Installed │ App Engine Go Extensions                             │ app-engine-go            │  4.2 MiB │
│ Not Installed │ Appctl                                               │ appctl                   │ 21.0 MiB │
│ Not Installed │ Cloud Bigtable Command Line Tool                     │ cbt                      │  9.8 MiB │
│ Not Installed │ C

In [12]:
! gcloud beta ai-platform versions create V2 \
  --model ChurnPredictor \
  --runtime-version 2.6 \
  --python-version 3.7 \
  --origin gs://aiplatform_irisgcp/ \
  --package-uris gs://aiplatform_irisgcp/custom_predict-0.1.tar.gz \
  --region global \
  --machine-type mls1-c4-m2 \
  --prediction-class predictor.ChurnPredictor 

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


In [13]:
instances=['768805383',45,'M','3','High School','Married','$60K - $80K','Blue',39,5,1,3,12691,777,11914,1.335,1144,42,1.625,0.061]

In [14]:
!gcloud ai-platform predict --model ChurnPredictor --region global --version V1 --json-request input.json

Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.predict) NOT_FOUND: Field: name Error: The specified model version was not found.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: The specified model version was not found.
    field: name
